In [169]:
#!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait

from time import sleep

from selenium.webdriver import Keys, ActionChains
from selenium.webdriver.common.actions.action_builder import ActionBuilder

from selenium.webdriver.common.desired_capabilities import DesiredCapabilities


import numpy as np

import pandas as pd

from selenium.webdriver.chrome.options import Options

## Choosing Some Products

In [29]:
df = pd.read_csv('https://raw.githubusercontent.com/RDeconomist/prices/master/data/db_item.csv')

Let's target 10 of these - food

In [33]:
target_ids = [210604,
 430605,
 210910,
 212701,
 212527,
 212601,
 212530,
 210107,
 211714,
 212901]

In [36]:
target_ids = [210604, 430605, 210910, 212701, 212527, 212601, 212530, 210107, 211714, 212901]
df[df['item_id'].isin(target_ids)]

,item_id,description,date_quote_s,date_quote_e,n_obs
4,210107,"BROWN LOAF,400G,SLICED-GRAN",198903,200401,29361
60,210604,IMP LAMB LOIN CHOP-BONE PER KG,199002,200901,25196
78,210910,FRESH BONELESS CHICKEN BREAST,200302,202303,57325
154,211714,NON-DAIRY MILK DRINK 900ML-1LT,201702,202303,16863
267,212527,PRE-PACKED SALAD 100-250G,200202,202303,21382
270,212530,FRESH VEG-BROCCOLI-PER KG,200702,202303,60640
276,212601,CANNED TOMATOES 390-400G,198802,202303,91215
284,212701,APPLES -COOKING-PER LB,198802,199812,37126
322,212901,CUSTARD POWDER-300G DRUM,198802,200001,22072
747,430605,CAT FOOD CAN 2,199601,199601,209


<h1> Scraping with Selenium </h1>
Selenium is a tool that automates browsers. It is mostly used for testing purposes, but it can also be used for web scraping. It's a more useful approach than simple requests+bs4 when the website is dynamic and requires javascript to load the content and when the sites try to block bots.

Let's try Tesco.

In [131]:
driver = webdriver.Firefox()

In [132]:

driver.get("https://www.tesco.com/groceries/en-GB/shop/home-and-ents/all?page=10&count=48/")


From snooping around the html code, we can see the best way in is to go by the href, and filter for products:

In [ ]:
def get_products(driver):
    """
    Args:
        driver: webdriver
    Returns:
        list of products
    """
    count = 0
    products = []
    a_s = driver.find_elements(By.TAG_NAME, "a")
    for a in a_s:
        # check if a has a href attribute
        if(a.get_attribute('href') != None and'/products/' in a.get_attribute('href')):
            product = {}
            count += 1
            #print(f"{count}:{a.get_attribute('href')}")
            try:
                span_child = a.find_element(By.TAG_NAME, "span")
                product = {'name': span_child.text, 'link': a.get_attribute('href')}
                gp = a.find_element(By.XPATH, '..')
                gp = gp.find_element(By.XPATH, '..')
                p_elements = gp.find_elements(By.TAG_NAME, "p")
                prices = []
                for p in p_elements:
                    if '£' in p.text:
                        prices.append(p.text)
                product['prices'] = prices
                products.append(product)
            except Exception as e:
                continue
    return products

        


Let's see how many we can do before they just block us.
We can try Fresh Food - the website tells us there are 3667 products in this category - so 77 pages of 48 products each.

In [154]:
urls = ["https://www.tesco.com/groceries/en-GB/shop/fresh-food/all/"]+[f"https://www.tesco.com/groceries/en-GB/shop/fresh-food/all/?page={i}&count=48" for i in range(2, 75)]

driver = webdriver.Firefox()
products = []
for url in urls:
    print(url)
    driver.get(url)
    page_products = get_products(driver)
    products += page_products
    if len(products) <5:
        break
    driver.implicitly_wait( np.clip(np.random.normal(1, 1), 0,2) )

https://www.tesco.com/groceries/en-GB/shop/fresh-food/all/?page=56&count=48
https://www.tesco.com/groceries/en-GB/shop/fresh-food/all/?page=57&count=48
https://www.tesco.com/groceries/en-GB/shop/fresh-food/all/?page=58&count=48
https://www.tesco.com/groceries/en-GB/shop/fresh-food/all/?page=59&count=48
https://www.tesco.com/groceries/en-GB/shop/fresh-food/all/?page=60&count=48
https://www.tesco.com/groceries/en-GB/shop/fresh-food/all/?page=61&count=48
https://www.tesco.com/groceries/en-GB/shop/fresh-food/all/?page=62&count=48
https://www.tesco.com/groceries/en-GB/shop/fresh-food/all/?page=63&count=48
https://www.tesco.com/groceries/en-GB/shop/fresh-food/all/?page=64&count=48
https://www.tesco.com/groceries/en-GB/shop/fresh-food/all/?page=65&count=48
https://www.tesco.com/groceries/en-GB/shop/fresh-food/all/?page=66&count=48
https://www.tesco.com/groceries/en-GB/shop/fresh-food/all/?page=67&count=48
https://www.tesco.com/groceries/en-GB/shop/fresh-food/all/?page=68&count=48
https://www.

In [157]:
import pickle as pkl
pkl.dump(products, open('product_pages.pkl', 'wb'))

Wow, so they didn't block at all. We were moving incredibly slowly though - 40 seconds for 48 products. Let's speed up!
We don't need to wait for the page to load - we can use 'eager' loading.

In [164]:
caps = DesiredCapabilities().FIREFOX
caps["pageLoadStrategy"] = "eager"  #  interactive
driver = webdriver.Firefox(desired_capabilities=caps)


/var/folders/by/095_7x8j04x77vcq4_rvm1_m0000gn/T/ipykernel_40103/2157608254.py:3: DeprecationWarning: capabilities and desired_capabilities have been deprecated, please pass in a Service object
  driver = webdriver.Firefox(desired_capabilities=caps)


In [166]:
driver.get(urls[3])
# wait 5 seconds
sleep(5)
page_products = get_products(driver)

AttributeError: 'builtin_function_or_method' object has no attribute 'sleep'

In [168]:
sleep(5)

NameError: name 'sleep' is not defined

In [161]:
len(page_products)

26

In [ ]:
https://www.tesco.com/groceries/en-GB/shop/fresh-food/all

In [135]:
3667/50

73.34

In [133]:
prods = get_products(driver)

In [134]:
prods

[{'name': 'Keep It Handy Reusable Shopping Bag',
  'link': 'https://www.tesco.com/groceries/en-GB/products/311935737',
  'prices': ['£1.50', '£1.50/each']},
 {'name': 'Fox & Ivy White Heart Serve Bowl Small',
  'link': 'https://www.tesco.com/groceries/en-GB/products/313801622',
  'prices': ['£2.00', '£2.00/each']},
 {'name': 'Gold Leaf Jps Quality Blend Tobacco 100G',
  'link': 'https://www.tesco.com/groceries/en-GB/products/311943998',
  'prices': ['£53.65', '£53.65/100g']},
 {'name': 'Signature Blue 5X10 Cigars',
  'link': 'https://www.tesco.com/groceries/en-GB/products/294349610',
  'prices': ['£33.13', '£0.66/each']},
 {'name': 'Mayfair Original Blue 5X20 Pack',
  'link': 'https://www.tesco.com/groceries/en-GB/products/310161907',
  'prices': ['£64.95', '£0.65/each']},
 {'name': 'Fox & Ivy Acacia 3 Bowl Serving Paddle',
  'link': 'https://www.tesco.com/groceries/en-GB/products/313664772',
  'prices': ['£12.00', '£12.00/each']},
 {'name': 'Benson & Hedges Silver Roll Your Own 30G',


In [102]:
a_s = driver.find_elements(By.TAG_NAME, "a")

In [103]:
count = 0
products = []
for a in a_s:
    # check if a has a href attribute
    if(a.get_attribute('href') != None and'/products/' in a.get_attribute('href')):
        product = {}
        count += 1
        #print(f"{count}:{a.get_attribute('href')}")
        try:
            span_child = a.find_element(By.TAG_NAME, "span")
            product = {'name': span_child.text, 'link': a.get_attribute('href')}
            gp = a.find_element(By.XPATH, '..')
            gp = gp.find_element(By.XPATH, '..')
            p_elements = gp.find_elements(By.TAG_NAME, "p")
            prices = []
            for p in p_elements:
                if '£' in p.text:
                    prices.append(p.text)
            product['prices'] = prices
            products.append(product)
        except Exception as e:
            continue

        

reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached
reached


In [104]:
len(products)

48

In [80]:
a = product_link_elements[1]

In [85]:
# Find a's grandparent
gp = a.find_element(By.XPATH, '..').find_element(By.XPATH, '..')
p_elements = gp.find_elements(By.TAG_NAME, "p")

In [87]:
for p in p_elements:
    print(p.text)

Please note that the products shown are an assortment, you could receive any one of the products shown, to make a request for a preference please use the picker notes available on checkout
£1.50
£1.50/each


In [83]:
gp.find_element(By.XPATH, '..')

<selenium.webdriver.remote.webelement.WebElement (session="ec5f6eea47a9e7c29fa26d00ecaea3e9", element="b20ac280-d3ce-427c-ac10-bfea60b31336")>

In [82]:
gp.tag_name

'h3'

In [65]:
product_link_elements[1].find_element(By.TAG_NAME, "span").text

'Keep It Handy Reusable Shopping Bag'

In [122]:
options = Options()
options.headless = True

chrome_options = webdriver.ChromeOptions()
# this will disable image loading
chrome_options.add_argument('--blink-settings=imagesEnabled=false')
chrome_options.add_argument("--headless")
# or alternatively we can set direct preference:
chrome_options.add_experimental_option(
    "prefs", {"profile.managed_default_content_settings.images": 2}
)

driver = webdriver.Chrome(options=options, chrome_options=chrome_options)

/var/folders/by/095_7x8j04x77vcq4_rvm1_m0000gn/T/ipykernel_40103/1629774272.py:2: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True
/var/folders/by/095_7x8j04x77vcq4_rvm1_m0000gn/T/ipykernel_40103/1629774272.py:13: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(options=options, chrome_options=chrome_options)


In [68]:
product_link_elements[0].find_element(By.TAG_NAME, "span")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"tag name","selector":"span"}
  (Session info: MicrosoftEdge=111.0.1661.51)
Stacktrace:
0   msedgedriver                        0x0000000104bb48a0 msedgedriver + 4376736
1   msedgedriver                        0x0000000104b46e30 msedgedriver + 3927600
2   msedgedriver                        0x00000001047c53fc msedgedriver + 250876
3   msedgedriver                        0x00000001047fa8e4 msedgedriver + 469220
4   msedgedriver                        0x00000001047f2670 msedgedriver + 435824
5   msedgedriver                        0x000000010482cad4 msedgedriver + 674516
6   msedgedriver                        0x00000001047f0c48 msedgedriver + 429128
7   msedgedriver                        0x00000001047f1dc8 msedgedriver + 433608
8   msedgedriver                        0x0000000104b89f94 msedgedriver + 4202388
9   msedgedriver                        0x0000000104b8dc88 msedgedriver + 4217992
10  msedgedr

In [45]:
driver.find_element(By.ID, "beans-masthead-desktop-search-input")ActionChains(driver)\
    .move_to_element(clickable)\

<selenium.webdriver.remote.webelement.WebElement (session="fb92853809aca9ea11ffd0a13faddddc", element="5708fd8c-150a-436e-bab3-f1391bc893f5")>

In [46]:

clickable = driver.find_element(By.ID, "beans-masthead-desktop-search-input")
ActionChains(driver)\
    .move_to_element(clickable)\
    .pause(np.random.normal(1))\
    .click()\
    .pause(np.random.normal(0.2))\
    .send_keys("Tomatoes")\
    .send_keys(Keys.ENTER)\
    .perform()

In [ ]:

clickable = driver.find_element(By.ID, "beans-masthead-desktop-search-input")
ActionChains(driver)\
    .move_to_element(clickable)\
    .pause(np.random.normal(1))\
    .click()\
    .pause(np.random.normal(0.2))\
    .send_keys("Tomatoes")\
    .send_keys(Keys.ENTER)\
    .perform()

In [3]:
def test_eight_components():
    driver = webdriver.Chrome()

    driver.get("https://www.selenium.dev/selenium/web/web-form.html")

    title = driver.title
    assert title == "Web form"

    driver.implicitly_wait(0.5)

    text_box = driver.find_element(by=By.NAME, value="my-text")
    submit_button = driver.find_element(by=By.CSS_SELECTOR, value="button")

    text_box.send_keys("Selenium")
    submit_button.click()

    message = driver.find_element(by=By.ID, value="message")
    value = message.text
    assert value == "Received!"

    driver.quit()

In [4]:
test_eight_components()

In [48]:
import requests

url = "https://www.tesco.com/groceries/en-GB/shop/home-and-ents/all?page=10&count=48"
req = requests.get(url)

KeyboardInterrupt: 